## Examples of types of interactions
In this note book we provide some examples of how to define local, nonlocal, single and multi-channel interactions.

In [1]:
import numpy as np

from jitr import reactions, rmatrix
from jitr.reactions.potentials import coulomb_charged_sphere, woods_saxon_potential
from jitr.utils import delta, kinematics

In [2]:
solver = rmatrix.Solver(nbasis=40)

In [3]:
# a local scalar interaction
def interaction_local(r, V0, W0, R0, a0, Zz):
    nuclear = woods_saxon_potential(r, V0, W0, R0, a0)
    coulomb = coulomb_charged_sphere(r, Zz, R0)
    return nuclear + coulomb

In [4]:
# a nonlocal scalar interaction
# note this exp(-(r-rp)**2) form is just made up for example purposes
# this is not the Perey-Buck form, which is actually l-dependent
def interaction_nonlocal(r, rp, V0, W0, R0, a0, Zz):
    nuclear = woods_saxon_potential(0.5 * (r + rp), V0, W0, R0, a0) * np.exp(
        (r - rp) ** 2
    )
    return nuclear

In [5]:
def interaction_2level(r, V0, W0, V1, W1, g, R0, a0, Zz):
    nuclear0 = woods_saxon_potential(r, V0, W0, R0, a0)
    nuclear1 = woods_saxon_potential(r, V1, W1, R0, a0)
    coupling = woods_saxon_potential(r, g * V0, 0, R0, a0)
    coulomb = coulomb_charged_sphere(r, Zz, R0)
    return np.array([[nuclear0 + coulomb, coupling], [coupling, nuclear1 + coulomb]])

In [6]:
def interaction_2level_nonlocal(r, rp, V0, W0, V1, W1, g, R0, a0, Zz):
    nuclear0 = woods_saxon_potential(0.5 * (r + rp), V0, W0, R0, a0) * np.exp(
        -((r - rp) ** 2)
    )
    nuclear1 = woods_saxon_potential(0.5 * (r + rp), V1, W1, R0, a0) * np.exp(
        -((r - rp) ** 2)
    )
    coupling = woods_saxon_potential(0.5 * (r + rp), g * V0, 0, R0, a0) * np.exp(
        -((r - rp) ** 2)
    )
    return np.array([[nuclear0, coupling], [coupling, nuclear1]])

In [7]:
# coupling between levels  within a partial wave
def coupling_2level(l):
    return np.array([[1, 1 / np.sqrt(2)], [1 / np.sqrt(2), 1]])

### Define the scattering system

In [8]:
Elab = 35  # MeV
Ca48 = (28, 20)
proton = (1, 1)

mass_target = kinematics.mass(*Ca48)
mass_projectile = kinematics.mass(*proton)
Zz = Ca48[1] * proton[1]

In [9]:
Ecm, mu, k, eta = kinematics.classical_kinematics(
    mass_target,
    mass_projectile,
    Elab,
    Zz,
)

In [10]:
# first the 1 level system
sys_1level = reactions.ProjectileTargetSystem(
    channel_radius=5 * np.pi,
    lmax=30,
    mass_target=mass_target,
    mass_projectile=mass_projectile,
    Ztarget=Ca48[1],
    Zproj=proton[1],
)
channels_1, asymptotics_1 = sys_1level.get_partial_wave_channels(Ecm, mu, k, eta)

In [11]:
# now the 2-level system
sys_2level = reactions.ProjectileTargetSystem(
    channel_radius=5 * np.pi,
    lmax=30,
    mass_target=mass_target,
    mass_projectile=mass_projectile,
    Ztarget=Ca48[1],
    Zproj=proton[1],
    coupling=coupling_2level,
)
channels_2, asymptotics_2 = sys_2level.get_partial_wave_channels(Ecm, mu, k, eta)

In [12]:
channels_2[0].couplings

array([[1.        , 0.70710678],
       [0.70710678, 1.        ]])

In [13]:
# grab some parameters out of a hat
V0 = 42.0
W0 = 18.1
R0 = 4.8
a0 = 0.7
V1 = V0
W1 = W0
g = 1 / np.sqrt(2)

# the single channel interactions are equivalent other than the non-locality
params_local = (V0, W0, R0, a0, Zz)
params_nonlocal = params_local

# the 2-level interactions  are likewise equivalent other than the non-locality
params_2level = (V0, W0, V1, W1, g, R0, a0, Zz)
params_2level_nonlocal = params_2level

# the non-local potentials also have a local Coulomb term
params_coul = (Zz, R0)

### Let's compare the 1-level systems

In [14]:
l = 0

In [15]:
R, S, uext_boundary = solver.solve(
    channels_1[l],
    asymptotics_1[l],
    local_interaction=interaction_local,
    local_args=params_local,
)
phase_shift, phase_attenuation = delta(S[0, 0])
print(f"phase shift: {phase_shift:1.3f} + i ({phase_attenuation:1.3f}) [degrees]")

phase shift: -56.596 + i (-32.962) [degrees]


In [16]:
R, S, uext_boundary = solver.solve(
    channels_1[l],
    asymptotics_1[l],
    local_interaction=coulomb_charged_sphere,
    local_args=params_coul,
    nonlocal_interaction=interaction_nonlocal,
    nonlocal_args=params_nonlocal,
)
phase_shift, phase_attenuation = delta(S[0, 0])
print(f"phase shift: {phase_shift:1.3f} + i ({phase_attenuation:1.3f}) [degrees]")

phase shift: -59.990 + i (0.047) [degrees]


### Let's compare the 2-level systems


In [17]:
R, S, uext_boundary = solver.solve(
    channels_2[l],
    asymptotics_2[l],
    local_interaction=interaction_2level,
    local_args=params_2level,
)
for i in range(2):
    for j in range(2):
        phase_shift, phase_attenuation = delta(S[i, j])
        print(
            f"({i},{j}): phase shift: {phase_shift} + i ({phase_attenuation}) [degrees]"
        )

(0,0): phase shift: -30.626543781068136 + i (-67.5676000526758) [degrees]
(0,1): phase shift: 63.39345247559675 + i (-69.75283143425965) [degrees]
(1,0): phase shift: 63.39345247559895 + i (-69.75283143426192) [degrees]
(1,1): phase shift: -30.62654378106169 + i (-67.56760005267303) [degrees]


In [18]:
R, S, uext_boundary = solver.solve(
    channels_2[l],
    asymptotics_2[l],
    nonlocal_interaction=interaction_2level_nonlocal,
    nonlocal_args=params_2level_nonlocal,
)
for i in range(2):
    for j in range(2):
        phase_shift, phase_attenuation = delta(S[i, j])
        print(
            f"({i},{j}): phase shift: {phase_shift} + i ({phase_attenuation}) [degrees]"
        )

(0,0): phase shift: -41.8967479737616 + i (-82.75501400242788) [degrees]
(0,1): phase shift: 50.374879789405774 + i (-80.93948141959028) [degrees]
(1,0): phase shift: 50.37487978940358 + i (-80.93948141959079) [degrees]
(1,1): phase shift: -41.896747973774204 + i (-82.75501400242803) [degrees]


## Let's compare speed

In [19]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_1[l],
    asymptotics_1[l],
    local_interaction=interaction_local,
    local_args=params_local,
)

492 µs ± 39.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_1[l],
    asymptotics_1[l],
    local_interaction=coulomb_charged_sphere,
    local_args=params_coul,
    nonlocal_interaction=interaction_nonlocal,
    nonlocal_args=params_nonlocal,
)

929 µs ± 77.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [21]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_2[l],
    asymptotics_2[l],
    local_interaction=interaction_2level,
    local_args=params_2level,
)

9.12 ms ± 2.09 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_2[l],
    asymptotics_2[l],
    nonlocal_interaction=interaction_2level_nonlocal,
    nonlocal_args=params_2level_nonlocal,
)

4.73 ms ± 2.26 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Let's see if `njit`-ing the interaction gives us any speed up.

In [23]:
# a local scalar interaction
from numba import njit


@njit
def interaction_local_njit(r, V0, W0, R0, a0, Zz):
    nuclear = woods_saxon_potential(r, V0, W0, R0, a0)
    return nuclear

In [24]:
# jit compile once
solver.solve(
    channels_1[l],
    asymptotics_1[l],
    local_interaction=interaction_local,
    local_args=params_local,
)

(array([[0.00313908-0.03434955j]]),
 array([[-1.24440751-2.90467644j]]),
 array([1.18181216-1.66335354j]))

In [25]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_1[l],
    asymptotics_1[l],
    local_interaction=interaction_local,
    local_args=params_local,
)

471 µs ± 70.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Answer: no